In [1]:
import numpy as np
import subprocess

In [61]:
year = 2019
teamCode = "CHN"
outputFile = "homeTeam2019.txt"

#subprocess.call(["bevent", "-h"], shell=True)
#subprocess.run(["bevent", "-f",  "1", "-y", str(year), str(year) + teamCode + ".EV?"], shell=True)
#subprocess.run(["bevent", "-f",  "1,2-4,8-9,26-28,78", "-y", str(year), str(year) + teamCode + ".EV?", "> FormattedData\\" + outputFile], shell=True)

# "bevent", "-f 1,2-4,8-9,26-28,78", "-y" + year + " " + year + teamAbr + ".EV?" > FormattedData\.txt

In [60]:
def createData(startYear, endYear, homeTeamCode, awayTeamCode): #setup files, note startYear, endYear is inclusive
    dataDirectory = "F:\\Users\\Daniel\\Machine Learning Work\\Baseball work\\test software"
    #get the away team files
    for year in range(startYear, endYear + 1):
        outputFileName = "homeTeam" + str(year) + ".txt"
        commandString = "bevent -f 2-4,8-9,26-28,78 -y " + str(year) + " " + str(year) + homeTeamCode + ".EV? > FormattedData\\" + outputFileName
        subprocess.run(commandString, cwd=dataDirectory,  shell=True)
    
    #get the home team files, requires modification afterwards
    for year in range(startYear, endYear + 1):
        outputFileName = "awayTeam" + str(year) + ".txt"
        commandString = "bevent -f 1,2-4,8-9,26-28,78 -y " + str(year) + " " + str(year) + "???.EV? > FormattedData\\" + outputFileName
        subprocess.run(commandString, cwd=dataDirectory,  shell=True)
    
    #delete prvious home and away files if they exist
    commandString = "del /f FormattedData\\homeTeam.txt"
    subprocess.run(commandString, cwd=dataDirectory,  shell=True)

    commandString = "del /f FormattedData\\awayTeam.txt"
    subprocess.run(commandString, cwd=dataDirectory,  shell=True)
    
    
    with open("test Software\\FormattedData\\homeTeam.txt", "w") as f: #create the complete home file, copy all the year files over
        for year in range(startYear, endYear + 1):
            outputFileName = "FormattedData\\homeTeam" + str(year) + ".txt"
            with open("test Software\\" + outputFileName, "r") as readFile:
                for line in readFile: #copy over the lines to the main file
                    f.write(line)
    
    with open("test Software\\FormattedData\\awayTeam.txt", "w") as f: #create the complete away file
        for year in range(startYear, endYear + 1):
            outputFileName = "FormattedData\\awayTeam" + str(year) + ".txt"
            with open("test Software\\" + outputFileName, "r") as readFile:
                for line in readFile: #copy over the lines to the main file
                    team = line[1:4] #away team from this game
                    if(team == awayTeamCode): #copy the conditionally
                        f.write(line[6:]) #copy the line but the team segment
         
    #remove the away and home team files generated, since they aren't needed anymore
    for year in range(startYear, endYear + 1):
        homeOutputFileName = "homeTeam" + str(year) + ".txt"
        awayOutputFileName = "awayTeam" + str(year) + ".txt"
        
        commandString = "del /f FormattedData\\" + homeOutputFileName
        subprocess.run(commandString, cwd=dataDirectory,  shell=True)
        
        commandString = "del /f FormattedData\\" + awayOutputFileName
        subprocess.run(commandString, cwd=dataDirectory,  shell=True)



In [92]:
#gameState represents any state in a baseball game, each state can be represented by either members or as one number
#note: end flags, they are inning = 0, outs = 0, bases = 0, score = 1,2,3 (since these states are impossible in normal play)
#if flag score is 1 (home win), 2 (away win), 3 (draw state at end of 17th (9th) inning)
class gameState:
    inning = 0 #[0,17] since each of the 9 innings are either in the top or the bottom
    outs = 0 #[0, 2] number of outs
    bases = 0 #[0,7] binary representation, i.e. 6 (110) has runners on 2nd and 3rd
    score = 0 #score = homeTeamScore - awayTeamScore, for simplicity score = [-15, 15]

    def setFromInfo(self, inn, out, bas, scor): #construct from data, note: base should be a 3 element list like [0,1,1]
        if(inn > 17):
            inn = 16 + int(inn % 2 == 0)
        self.inning = inn
        self.outs = out
        self.bases = 4*bas[2] + 2*bas[1] + bas[0] #conversion from list format to single number
        
        self.score = max(-15, min(15, scor)) #score between -15 and 15
        
    def setFromData(self, inn, out, bas, scor): #construct from data, note: base should be [0,7]
        if(inn > 17):
            inn = 16 + int(inn % 2 == 0)
        self.inning = inn
        self.outs = out
        self.bases = bas
        
        self.score = max(-15, min(15, scor)) #score between -15 and 15

    def setFromNum(self, numEquiv): #construct from number, map number to gameState
        self.outs = numEquiv % 3
        numEquiv = numEquiv // 3
        
        self.bases = numEquiv % 8
        numEquiv = numEquiv // 8
        
        self.inning = (numEquiv % 18)
        numEquiv = numEquiv // 18
        
        self.score = (numEquiv % 31) - 15
        
    def __str__(self): #for convient printing, note bases are printed as 1,2,3 not 3,2,1
        if(self.isEndState()): #print in special format if an end game state
            if(self.score == 2):
                return "{Away win}"
            elif(self.score == 1):
                return "{Home win}"
            else:
                return "{Draw}"   
        return ("{'Inning': " + str(self.inning) + ", " 
            + "'Outs': " + str(self.outs) + ", "
            + "'Bases': " + str(self.bases % 2) + str((self.bases >> 1) % 2) + str(self.bases >> 2) + ", "
            + "'Score': " + str(self.score) + "}")
    
    def toNum(self): #map gameState to number
            return (self.score + 15)*18*8*3 + (self.inning)*8*3 + (self.bases)*3 + (self.outs)
    
    def setEndFlagState(self, lastState): #set the end flag state from the last state of the game, asssume lastState is an end
        self.inning = 0
        self.outs = 0
        self.bases = 0
        self.score = 3 #draw
        if(lastState.score < 0): #away win
            self.score = 2
        elif(lastState.score > 0): #home win
            self.score = 1
        else: #game ended in overtime and scores are equal
            self.score = 1 #is this correct
                     
    def isEndState(self): #check if the state is an end state
        if(self.inning != 0 or self.outs != 0 or self.bases != 0): #not an end state if any of these aren't true
            return False
        return (self.score >= 1 and self.score <= 3) #is an end flag if score is between [1, 3]
    def getEndResult(self): #return the value of an end state 1 = home win, -1 away win, 0 draw (assumes the state is an end state)        
        if(self.score == 1):
            return 1 #home win
        elif(self.score == 2):
            return -1 #away win
        else:
            return 0 #draw
        
    def getStateSpace(self): #return the number of possible states
        return 3*8*18*31
#gameStates options: 18*3*8*31 = 13392

def getStateFromLine(line): #line in format of: inning, top/bottom, out, vis. score, home score, 1st,2nd,3rd, newGameFlag
    lineData = line.split(',')
    
    #convert from the line to the state
    inning = (int(lineData[0])-1)*2 + int(lineData[1]) #map inning and bottom/top for state
    outs = int(lineData[2])
    bases = [int(lineData[5] != '""'), int(lineData[6] != '""'), int(lineData[7] != '""')] #convert to bases
    score = int(lineData[4]) - int(lineData[3]) #home - vis score
    
    newGameStart = lineData[8][1] == 'T' #weird way of checking string, but it works
    
    state = gameState()
    state.setFromInfo(inning,outs,bases,score)
    return state, newGameStart

def getFileCount(fileName): #get the counted matrix from the file
    dataFile = open(fileName, "r")

    stateCurr = gameState()
    statePrev = gameState()
    stateEnd = gameState()
    
    matrixDim = (stateCurr.getStateSpace(),stateCurr.getStateSpace()) #since 31*18*8*3
    matr = np.zeros(matrixDim) #set the size of the large state transition matrix

    line = dataFile.readline()

    gameCount = 1 #1 for initial starting game

    statePrev, newGame = getStateFromLine(line) #avoid running into an error of writing the last game end
    line = dataFile.readline() #read second line, not a newGame
    while(line):
        stateCurr, newGame = getStateFromLine(line)
        if(not newGame):
            matr[statePrev.toNum()][stateCurr.toNum()] += 1 #add one to the entry
        else: #new game, put result of previous game
            stateEnd.setEndFlagState(statePrev) #get the flag from the end state
            #print(lineNum, stateEnd, statePrev)
            matr[statePrev.toNum()][stateEnd.toNum()] += 1 #add one to the end state entry
            gameCount = gameCount + 1

        statePrev = stateCurr
        line = dataFile.readline()

    dataFile.close()
    print("Total games in file: ", gameCount)
    
    return matr
    
def getTransMatrix(countMatrix): #go from count matrix to transition matrix

    matrixDim = np.shape(countMatrix)
    stateTran = np.zeros(matrixDim) #set the size of the large state transition matrix

    #win coordinates should have only 1 entry in their row, gaurenteed chance to return to iteself
    stateEnd = gameState()
    
    stateEnd.setFromData(0,0,0,1) #home win
    countMatrix[stateEnd.toNum()] = np.zeros(matrixDim[1]) #set to zeros to remove any "bad" data
    stateEnd.setFromData(0,0,0,2) #home loss
    countMatrix[stateEnd.toNum()] = np.zeros(matrixDim[1]) #set to zeros to remove any "bad" data
    stateEnd.setFromData(0,0,0,3) #draw
    countMatrix[stateEnd.toNum()] = np.zeros(matrixDim[1]) #set to zeros to remove any "bad" data
    
    #change the matrix to a probability matrix, i.e. matrix[y][x] is probability from going to x, given that you are at y
    for row in range(matrixDim[0]):
        rowSum = sum(countMatrix[row])
        if(rowSum != 0): 
            stateTran[row] = countMatrix[row] / rowSum
        else: #sum is zero, if any other rows lead here, set diagonal to 1
            stateTran[row][row] = 1

    return stateTran

In [109]:
#setup parameters for creating data files from bevent
teamCodeList = ["ANA","BAL","BOS","CHA","CLE","DET","HOU","KCA","MIN","NYA","OAK","SEA","TBA","TEX","TOR",
                "ARI","ATL","CHN","CIN","COL","LAN","MIA","MIL","NYN","PHI","PIT","SDN","SFN","SLN","WAS"]

minDate = 2000
maxDate = 2019

startDate = 0
endDate = 0
while(startDate > maxDate or startDate < minDate): #validate data
    startDate = int(input("Enter start date: "))
while(endDate > maxDate or endDate < minDate): #validate data
    endDate = int(input("Enter end date: "))
homeTeamCode = ""
awayTeamCode = ""
while(not homeTeamCode in teamCodeList):
    homeTeamCode = input("Enter home team code: ")
while(not awayTeamCode in teamCodeList):
    awayTeamCode = input("Enter away team code: ")

Enter start date: 2015
Enter end date: 2016
Enter home team code: CHN
Enter away team code: CHA


In [110]:
createData(startDate, endDate, homeTeamCode, awayTeamCode)

homeCount = getFileCount("F:\\Users\\Daniel\\Machine Learning Work\\Baseball work\\test software\\FormattedData\\homeTeam.txt")
homeTran = getTransMatrix(homeCount)
awayCount = getFileCount("F:\\Users\\Daniel\\Machine Learning Work\\Baseball work\\test software\\FormattedData\\awayTeam.txt")
awayTran = getTransMatrix(awayCount)

stanTran = getTransMatrix(getFileCount("F:\\Users\\Daniel\\Machine Learning Work\\Baseball work\\test software\\FormattedData\\2000-2019StateData.txt"))

Total games in file:  162
Total games in file:  162
Total games in file:  48588


In [71]:
def regularizeRow(dataRow, dataCount, standardRow):
    regRow = np.ones(np.shape(dataRow))
    for i in range(len(dataRow)):
        if(dataCount[i] != 0): #already correct value if there are no data points
            regRow[i] = 1/np.sqrt(dataCount[i] + 1)
    
    newRow = ( regRow*standardRow + (1-regRow)*dataRow )
    
    if(sum(newRow) != 0):
        newRow = newRow/sum(newRow) #normalize row since there may be some loss
    return newRow

def mergeMatrix(home, away): #merge a home and away probability matrix into one, can be used to get match off win probability
    s = gameState() #move through all states and 
    m = np.zeros(home.shape)
    
    for n in range(m.shape[0]):
        s.setFromNum(n)
        if(s.inning % 2 == 0): #away team at bat
            m[n] = away[n]
        else: #away team at bat
            m[n] = home[n]
    return m

In [111]:
#reguralize home and away matrices
homeRegTran = np.zeros(np.shape(homeTran))
awayRegTran = np.zeros(np.shape(awayTran))

homeVsAwayCount = mergeMatrix(homeCount,awayCount)
homeVsAwayTran = mergeMatrix(homeTran,awayTran)
homeVsAwayRegTran = np.zeros(np.shape(homeVsAwayTran))
for i in range(len(homeVsAwayRegTran)):
    homeVsAwayRegTran[i] = regularizeRow(homeVsAwayTran[i], homeVsAwayCount[i], stanTran[i])

In [112]:
winRegMatrix = np.linalg.matrix_power(homeVsAwayRegTran, 512) #almost any game should be over by 512 states or "plays"
#look at sparse matrix
#winMatrix = np.linalg.matrix_power(homeVsAwayTran, 512) #almost any game should be over by 512 states or "plays"

startState = gameState()
endState = gameState()
startState.setFromData(0,0,0,0)
y = startState.toNum()

print("With reg:")
for x in range(np.shape(winRegMatrix)[1]):
    if(winRegMatrix[y][x] > .0001):
        endState.setFromNum(x)
        print(endState, ", chance: ", winRegMatrix[y][x])

With reg:
{Home win} , chance:  0.5445854550366125
{Away win} , chance:  0.4554145449633856


In [113]:
#shows how regularization changes values
testState = gameState()
testState2 = gameState()
testState.setFromData(3,0,0,1)
print(testState)
print()

print("standard: ")
for i in range(len(stanTran[testState.toNum()])):
    if(stanTran[testState.toNum()][i] != 0):
        testState2.setFromNum(i)
        print(testState2, stanTran[testState.toNum()][i])
print("\n")

print("data: ")
for i in range(len(homeVsAwayTran[testState.toNum()])):
    if(homeVsAwayTran[testState.toNum()][i] != 0):
        testState2.setFromNum(i)
        print(testState2, homeVsAwayTran[testState.toNum()][i], homeVsAwayCount[testState.toNum()][i])
print("\n")
        
print("Reg: ")
for i in range(len(homeVsAwayRegTran[testState.toNum()])):
    if(homeVsAwayRegTran[testState.toNum()][i] != 0):
        testState2.setFromNum(i)
        print(testState2, homeVsAwayRegTran[testState.toNum()][i])

{'Inning': 3, 'Outs': 0, 'Bases': 000, 'Score': 1}

standard: 
{'Inning': 3, 'Outs': 0, 'Bases': 000, 'Score': 1} 0.00046490004649000463
{'Inning': 3, 'Outs': 1, 'Bases': 000, 'Score': 1} 0.6781342011467535
{'Inning': 3, 'Outs': 0, 'Bases': 100, 'Score': 1} 0.233224856655819
{'Inning': 3, 'Outs': 0, 'Bases': 010, 'Score': 1} 0.04726483805981714
{'Inning': 3, 'Outs': 0, 'Bases': 001, 'Score': 1} 0.005113900511390051
{'Inning': 3, 'Outs': 0, 'Bases': 000, 'Score': 2} 0.03579730357973036


data: 
{'Inning': 3, 'Outs': 1, 'Bases': 000, 'Score': 1} 0.75 12.0
{'Inning': 3, 'Outs': 0, 'Bases': 100, 'Score': 1} 0.1875 3.0
{'Inning': 3, 'Outs': 0, 'Bases': 000, 'Score': 2} 0.0625 1.0


Reg: 
{'Inning': 3, 'Outs': 0, 'Bases': 000, 'Score': 1} 0.00044835899678398694
{'Inning': 3, 'Outs': 1, 'Bases': 000, 'Score': 1} 0.7040923412457665
{'Inning': 3, 'Outs': 0, 'Bases': 100, 'Score': 1} 0.2028777756385376
{'Inning': 3, 'Outs': 0, 'Bases': 010, 'Score': 1} 0.045583164673038676
{'Inning': 3, 'Outs': 